# Sample

Sample a finetuned model, hopefully with tool usage.

In [ ]:
#| default_exp inference

In [ ]:
#| export
from __future__ import annotations
import math, random, torch, matplotlib.pyplot as plt, numpy as np, matplotlib as mpl, shutil, os, gzip, pickle, re, copy, time
from pathlib import Path
from functools import partial
import fastcore.all as fc
from glob import glob
import json

from torch import tensor, nn, optim
import torch.nn.functional as F
from datasets import load_dataset
from torch.optim import lr_scheduler
from torch.utils.data import DataLoader, default_collate
from torch.nn import init
from torch.nn.utils.rnn import pad_sequence
from typing import List, Optional

from datetime import datetime, timedelta
import calendar
from fastprogress import progress_bar
from einops import rearrange

from toolformer.datasets import *
from toolformer.tokenizer import *
from toolformer.model import *
from toolformer.tools import *
from toolformer.filtering import *

### Load model

In [ ]:
#| export
def load_finetuned_model(llama_path:str, tokenizer, local_rank:int, world_size:int, lora_path:str=None) -> Transformer:
    checkpoints = sorted(Path(llama_path).glob("*.pth"))
    assert (
        world_size == len(checkpoints)
    ), f"Loading a checkpoint for MP={len(checkpoints)} but world size is {world_size}"
    ckpt_path = checkpoints[local_rank]
    print("Loading...")
    checkpoint = torch.load(ckpt_path, map_location="cpu")
    
    with open(Path(llama_path) / "params.json", "r") as f:
        params = json.loads(f.read())
    
    model_args: ModelArgs = ModelArgs(max_seq_len=2048, max_batch_size=8, **params)
    model_args.vocab_size = tokenizer.n_words
    if lora_path: model_args.lora = True
    torch.set_default_tensor_type(torch.cuda.HalfTensor)
    model = Transformer(model_args).cuda().half()
    torch.set_default_tensor_type(torch.FloatTensor)
    
    model.load_state_dict(checkpoint, strict=False)
    if lora_path: 
        lora_weights = torch.load(lora_path)
        model.load_state_dict(lora_weights, strict=False)
    return model

In [ ]:
#| export
@torch.no_grad()
def sample_with_tools(
    model,
    tokenizer,
    prompts: List[str],
    max_gen_len: int,
    temperature: float = 0.8,
    top_p: float = 0.95,
    decode:bool=False,
    device:str='cuda', 
    top_k:int=10,
    api_start_token=20577,
    api_end_token=6580
) -> List[str]:
    
    bsz = len(prompts)
    params = model.params
    assert bsz <= params.max_batch_size, (bsz, params.max_batch_size)
    if make_api_calls: assert len(prompts) == 1

    prompt_tokens = [tokenizer.encode(x, bos=True, eos=False) for x in prompts]

    min_prompt_size = min([len(t) for t in prompt_tokens])
    max_prompt_size = max([len(t) for t in prompt_tokens])

    total_len = min(params.max_seq_len, max_gen_len + max_prompt_size)
    # import pdb; pdb.set_trace()
    tokens = torch.zeros(bsz, total_len).to(device).long()
    for k, t in enumerate(prompt_tokens):
        tokens[k, : len(t)] = torch.tensor(t).long()
    input_text_mask = tokens != tokenizer.pad_id
    start_pos = min_prompt_size
    prev_pos = 0
    inside_api_call = False
    l = 0
    
    for cur_pos in range(start_pos, total_len):
        logits = model.forward(tokens[:, prev_pos:cur_pos], prev_pos, last_logits_only=True)
        if temperature > 0:
            probs = torch.softmax(logits / temperature, dim=-1)
            probs_topk = probs.topk(dim=-1, k=10)
            if not inside_api_call and api_start_token in probs_topk.indices:
                next_token = api_start_token
                inside_api_call = True
            else: next_token = sample_top_p(probs, top_p)  
        else:
            next_token = torch.argmax(logits, dim=-1)
        next_token = next_token.reshape(-1)
        # only replace token if prompt has already been generated
        # next_token = torch.where(
        #     input_text_mask[:, cur_pos], tokens[:, cur_pos], next_token
        # )
        tokens[:, cur_pos] = next_token
        if make_api_calls and inside_api_call and next_token == api_end_token:
            prompt = "".join(self.decode(tokens))
            prompt_with_api_responses = make_api_calls([prompt])
            tokens_with_api_responses = toolformer.encode_to_tensor(prompt)
            tokens[0,:len(tokens_with_api_responses)] = tokens_with_api_responses
            l = len(tokens_with_api_responses) - cur_pos
            inside_api_call = False
        cur_pos = cur_pos + l
        prev_pos = cur_pos

    return tokens if not decode else decode_tokens(tokenizer, tokens, prompt_tokens, max_gen_len)

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()

InterpolationMissingOptionError: Bad value substitution: option 'lib_name' in section 'DEFAULT' contains an interpolation key 'repo' which is not a valid option name. Raw value: '%(repo)s'

### Test

In [ ]:
# import json

In [ ]:
# path = '/home/models/foundation/LLaMA/tokenizer.model'
# tokenizer = Tokenizer(path)

In [ ]:
# os.environ['LOCAL_RANK'] = '0'
# os.environ['WORLD_SIZE'] = '1'
# os.environ['RANK'] = '0'
# os.environ['MASTER_ADDR'] = '172.17.0.3'
# os.environ['MASTER_PORT'] = '6006'

In [ ]:
# local_rank, world_size = setup_model_parallel()

In [ ]:
# llama_path = '/home/models/foundation/LLaMA/7B/'
# lora_path = '/home/libs/toolformer/models/toolformer_7b_weights_part_0.pth'
# model = load_finetuned_model(llama_path, tokenizer, local_rank, world_size, lora_path=lora_path)

In [ ]:
# prompt = 'What month will it be in 5 weeks time?'
# sample_with_tools(model, tokenizer, [prompt], 100, decode=True)